In [138]:
import pandas as pd
import numpy as np

In [139]:
file_path = r"D:\ML\Projects-20240722T093004Z-001\Projects\vehicle_price_prediction\Vehicle Price Prediction\dataset.csv"
vehical_data = pd.read_csv(file_path)   
vehical_data.head()  # Display the first few rows of the dataset

df = pd.DataFrame(vehical_data)


In [140]:
df.head()

,name,description,make,model,year,price,engine,cylinders,fuel,mileage,transmission,trim,body,doors,exterior_color,interior_color,drivetrain
0,2024 Jeep Wagoneer Series II,"\n \n Heated Leather Seats, Nav Sy...",Jeep,Wagoneer,2024,74600.0,24V GDI DOHC Twin Turbo,6.0,Gasoline,10.0,8-Speed Automatic,Series II,SUV,4.0,White,Global Black,Four-wheel Drive
1,2024 Jeep Grand Cherokee Laredo,Al West is committed to offering every custome...,Jeep,Grand Cherokee,2024,50170.0,OHV,6.0,Gasoline,1.0,8-Speed Automatic,Laredo,SUV,4.0,Metallic,Global Black,Four-wheel Drive
2,2024 GMC Yukon XL Denali,NaN,GMC,Yukon XL,2024,96410.0,"6.2L V-8 gasoline direct injection, variable v...",8.0,Gasoline,0.0,Automatic,Denali,SUV,4.0,Summit White,Teak/Light Shale,Four-wheel Drive
3,2023 Dodge Durango Pursuit,White Knuckle Clearcoat 2023 Dodge Durango Pur...,Dodge,Durango,2023,46835.0,16V MPFI OHV,8.0,Gasoline,32.0,8-Speed Automatic,Pursuit,SUV,4.0,White Knuckle Clearcoat,Black,All-wheel Drive
4,2024 RAM 3500 Laramie,\n \n 2024 Ram 3500 Laramie Billet...,RAM,3500,2024,81663.0,24V DDI OHV Turbo Diesel,6.0,Diesel,10.0,6-Speed Automatic,Laramie,Pickup Truck,4.0,Silver,Black,Four-wheel Drive


In [141]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor



In [142]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1002 entries, 0 to 1001
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            1002 non-null   object 
 1   description     946 non-null    object 
 2   make            1002 non-null   object 
 3   model           1002 non-null   object 
 4   year            1002 non-null   int64  
 5   price           979 non-null    float64
 6   engine          1000 non-null   object 
 7   cylinders       897 non-null    float64
 8   fuel            995 non-null    object 
 9   mileage         968 non-null    float64
 10  transmission    1000 non-null   object 
 11  trim            1001 non-null   object 
 12  body            999 non-null    object 
 13  doors           995 non-null    float64
 14  exterior_color  997 non-null    object 
 15  interior_color  964 non-null    object 
 16  drivetrain      1002 non-null   object 
dtypes: float64(4), int64(1), object(1

In [143]:
df.isnull().sum().sort_values(ascending=False)

cylinders         105
description        56
interior_color     38
mileage            34
price              23
fuel                7
doors               7
exterior_color      5
body                3
engine              2
transmission        2
trim                1
name                0
model               0
make                0
year                0
drivetrain          0
dtype: int64

In [144]:
df.columns

Index(['name', 'description', 'make', 'model', 'year', 'price', 'engine',
       'cylinders', 'fuel', 'mileage', 'transmission', 'trim', 'body', 'doors',
       'exterior_color', 'interior_color', 'drivetrain'],
      dtype='object')

In [145]:
df.shape

(1002, 17)

In [190]:
# Separate features and target
X = df.drop('price', axis=1)
y = df['price']

X.shape, y.shape


((1002, 16), (1002,))

In [191]:
y.isnull().sum()

np.int64(23)

In [199]:
# Impute missing values in target column
target_imputer = SimpleImputer(strategy='mean')
y = target_imputer.fit_transform(y.values.reshape(-1, 1)).ravel()


AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [ ]:
print("Missing values in y:", np.isnan(y).sum())  # Should be 0

Missing values in y: 0


In [201]:
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols

['year', 'cylinders', 'mileage', 'doors']

In [202]:
categorical_cols

['name',
 'description',
 'make',
 'model',
 'engine',
 'fuel',
 'transmission',
 'trim',
 'body',
 'exterior_color',
 'interior_color',
 'drivetrain']

In [203]:
df[numerical_cols].head()

,year,cylinders,mileage,doors
0,2024,6.0,10.0,4.0
1,2024,6.0,1.0,4.0
2,2024,8.0,0.0,4.0
3,2023,8.0,32.0,4.0
4,2024,6.0,10.0,4.0


In [204]:
df[categorical_cols].head()

,name,description,make,model,engine,fuel,transmission,trim,body,exterior_color,interior_color,drivetrain
0,2024 Jeep Wagoneer Series II,"\n \n Heated Leather Seats, Nav Sy...",Jeep,Wagoneer,24V GDI DOHC Twin Turbo,Gasoline,8-Speed Automatic,Series II,SUV,White,Global Black,Four-wheel Drive
1,2024 Jeep Grand Cherokee Laredo,Al West is committed to offering every custome...,Jeep,Grand Cherokee,OHV,Gasoline,8-Speed Automatic,Laredo,SUV,Metallic,Global Black,Four-wheel Drive
2,2024 GMC Yukon XL Denali,NaN,GMC,Yukon XL,"6.2L V-8 gasoline direct injection, variable v...",Gasoline,Automatic,Denali,SUV,Summit White,Teak/Light Shale,Four-wheel Drive
3,2023 Dodge Durango Pursuit,White Knuckle Clearcoat 2023 Dodge Durango Pur...,Dodge,Durango,16V MPFI OHV,Gasoline,8-Speed Automatic,Pursuit,SUV,White Knuckle Clearcoat,Black,All-wheel Drive
4,2024 RAM 3500 Laramie,\n \n 2024 Ram 3500 Laramie Billet...,RAM,3500,24V DDI OHV Turbo Diesel,Diesel,6-Speed Automatic,Laramie,Pickup Truck,Silver,Black,Four-wheel Drive


In [205]:
df[categorical_cols].isnull().sum().sort_values(ascending=False)

description       56
interior_color    38
fuel               7
exterior_color     5
body               3
engine             2
transmission       2
trim               1
model              0
make               0
name               0
drivetrain         0
dtype: int64

In [206]:
df[numerical_cols].isnull().sum().sort_values(ascending=False)

cylinders    105
mileage       34
doors          7
year           0
dtype: int64

In [207]:
# Preprocessing pipeline
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

numeric_transformer

Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

In [208]:
categorical_transformer

Pipeline(steps=[('imputer',
                 SimpleImputer(fill_value='missing', strategy='constant')),
                ('onehot',
                 OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

In [209]:
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['year', 'cylinders', 'mileage', 'doors']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 ['name', 'description', 'make', 'model',
                                  'engine', 'fuel', 'transmission', 'trim',
                                  'body', 'exterior_color', 'interior_color',
                                  'drivetrain'])])

In [210]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [211]:
y_train

array([ 43000.        ,  95230.        ,  67784.        ,  47451.        ,
        39997.        ,  25311.        ,  28265.        ,  45780.        ,
        41720.        ,  89450.        ,  60200.        ,  25311.        ,
       108537.        ,  35913.        ,  54585.        ,  76300.        ,
        47340.        ,  50486.        ,  21889.        ,  90350.        ,
        88478.        ,  35195.        ,  44195.        ,  46246.        ,
        96410.        ,  32406.        ,  72357.        ,  31805.        ,
        50202.98569969,  47165.        ,  69315.        ,  46775.        ,
        77522.        ,  49820.        ,  53595.        ,  46835.        ,
        33668.        ,  49795.        ,  36882.        ,  46835.        ,
        54949.        ,  50945.        ,  34572.        ,  83910.        ,
        42005.        ,  87190.        ,  51645.        ,  49390.        ,
        56345.        ,  41180.        ,  31610.        ,  54800.        ,
        50202.98569969,  

In [222]:
# Linear Regression
lr_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', LinearRegression())])
lr_pipeline.fit(X_train, y_train)
y_preds = lr_pipeline.predict(X_test)

lr_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['year', 'cylinders',
                                                   'mileage', 'doors']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['name', 'description',
                                                   'make', 'model', 'engine',
                                                   'fuel', 'transmission',
                                                   'trim', 'body',
                                                   'exterior_color',
                                                   'interior_color',
                                                   'drivetrain'])])),
                ('model', LinearRegression())])

In [225]:
def evaluate_regression_model(y_test, y_preds, model_name="Model"):
    rmse = np.sqrt(mean_squared_error(y_test, y_preds))
    mae = mean_absolute_error(y_test, y_preds)
    r2 = r2_score(y_test, y_preds)
    
    print(f"📊 Evaluation Metrics for {model_name}:")
    print(f"✅ RMSE: {rmse:.4f}")
    print(f"✅ MAE : {mae:.4f}")
    print(f"✅ R²  : {r2:.4f}")
    
    return {"RMSE": rmse, "MAE": mae, "R2": r2}

In [226]:
metrics = evaluate_regression_model(y_test, y_preds, model_name="lr_pipeline")

📊 Evaluation Metrics for lr_pipeline:
✅ RMSE: 11548.8805
✅ MAE : 5278.9151
✅ R²  : 0.4768
